In [0]:
!pip install tensorflow==2.0.0-alpha0 --upgrade
!pip install tensorflow-gpu==2.0.0-alpha0 --upgrade

In [0]:
from __future__ import absolute_import, division, print_function

from datetime import datetime
import os
from shutil import copyfile

import tensorflow as tf
print('Tensorflow Version: ', tf.__version__)
print('GPU: ', tf.test.gpu_device_name())
from tensorflow.compat.v1.keras import Sequential, layers, models
import numpy as np

from google.colab import drive

drive.mount('/content/gdrive')
drive_dest_folder = '/content/gdrive/My Drive/Colab Notebooks/data'

# MNIST Dataset parameters.
pixel_res = 28
num_features = pixel_res * pixel_res # data features (img shape: 28*28).

# Network parameters.
noise_dim = num_features # Noise data points.

In [0]:
def generator_model():
    model = Sequential()
    model.add(layers.Dense(4096, input_shape=(noise_dim,)))
    model.add(layers.Activation('tanh'))
    model.add(layers.Dense(128*7*7))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('tanh'))
    model.add(layers.Reshape((7, 7, 128), input_shape=(128*7*7,)))
    model.add(layers.UpSampling2D(size=(2, 2)))
    model.add(layers.Conv2D(64, (5, 5), padding='same'))
    model.add(layers.Activation('tanh'))
    model.add(layers.UpSampling2D(size=(2, 2)))
    model.add(layers.Conv2D(1, (5, 5), padding='same'))
    model.add(layers.Activation('tanh'))
    return model


def discriminator_model():
    model = Sequential()
    model.add(layers.Reshape(
      target_shape=(pixel_res, pixel_res, 1),
      input_shape=(pixel_res, pixel_res, 1)))
    model.add(
      layers.Conv2D(64, (5, 5), padding='same')
    )
    model.add(layers.Activation('tanh'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Conv2D(128, (5, 5)))
    model.add(layers.Activation('tanh'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(1024))
    model.add(layers.Activation('tanh'))
    model.add(layers.Dense(1))
    model.add(layers.Activation('sigmoid'))
    return model


def create_combination(gen, dis):
  new_model = tf.keras.Sequential()
  new_model.add(gen)
  dis.trainable = False
  new_model.add(dis)
  return new_model

In [0]:
def dated_filename():
  return str(datetime.now().strftime('%Y_%m_%d_%H_%M_%S'))


def save_model(model, name):
  blob_name = '{0}_{1}.h5'.format(name, dated_filename())
  print('Saving -> {}'.format(blob_name))
  model.save(os.path.join(drive_dest_folder, blob_name))


def load_latest_model(name):
  model_files = [
      f
      for f in os.listdir(drive_dest_folder)
      if name in f and 'h5' in f
  ]
  model_files.sort()
  latest_file_name = model_files[-1]
  latest_filepath = os.path.join(drive_dest_folder, latest_file_name)
  print('Loading -> {}'.format(latest_filepath))
  model = tf.keras.models.load_model(latest_filepath)
  return model

In [0]:
loading = True
if loading:
  generator = load_latest_model('generator')
  discriminator = load_latest_model('discriminator')
else:
  generator = generator_model()
  discriminator = discriminator_model()

print(generator.output_shape)
print(discriminator.input_shape)
combined = create_combination(generator, discriminator)
total_training_epochs = 0

In [0]:
# Training parameters.
learning_rate = 0.0001
batch_size = 128

In [0]:
generator.compile(
    loss='binary_crossentropy',
#     optimizer="SGD"
    optimizer=tf.keras.optimizers.SGD(lr=learning_rate)
#     optimizer=tf.keras.optimizers.Adam(learning_rate)
)

discriminator.trainable = False
combined.compile(
    loss='binary_crossentropy',
    optimizer=tf.keras.optimizers.SGD(
        lr=learning_rate, momentum=0.9, nesterov=True
    )
#     optimizer=tf.keras.optimizers.Adam(learning_rate)
)

discriminator.trainable = True
discriminator.compile(
    loss='binary_crossentropy',
    optimizer=tf.keras.optimizers.SGD(
        lr=learning_rate, momentum=0.9, nesterov=True
    )
#     optimizer=tf.keras.optimizers.Adam(learning_rate)
)

true_labels = tf.ones([batch_size], dtype=tf.float32)
false_labels = tf.zeros([batch_size], dtype=tf.float32)

In [0]:

# Prepare MNIST data.
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# Convert to float32.
x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)
# Normalize images value from [0, 255] to [0, 1].
x_train, x_test = x_train / 255., x_test / 255.
# reshape for the disciminator
x_train = np.reshape(x_train, (-1, pixel_res, pixel_res, 1))
x_test = np.reshape(x_test, (-1, pixel_res, pixel_res, 1))
# Rescale to [-1, 1], the input range of the discriminator
x_train = (x_train * 2) - 1
x_test = (x_test * 2) - 1

# Use tf.data API to shuffle and batch data.
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().shuffle(60000).batch(batch_size).prefetch(1)

In [0]:
# Visualize predictions.
import matplotlib.pyplot as plt


def generate_sliding_noise(indexes_to_slide=[0]):
  noise_line = np.random.normal(-1., 1., size=noise_dim).astype(np.float32)
  noise = np.zeros([36, noise_dim]).astype(np.float32)
  for i in range(noise.shape[0]):
    noise[i] = noise_line
    for n in indexes_to_slide:
      noise[i][n] = ((i / (noise.shape[0] - 1)) * 2) - 1
  return noise


def draw_data(t):
  # Rescale to original [0, 1]
  t = (t + 1.) / 2
  # Reverse colors for better display
  t = -1 * (t - 1)
  canvas = np.empty((pixel_res * n, pixel_res * n))
  for i in range(n):
      for j in range(n):
          image_data = t[(i * n) + j].reshape([pixel_res, pixel_res])
          canvas[i * pixel_res:(i + 1) * pixel_res, j * pixel_res:(j + 1) * pixel_res] = image_data
  plt.figure(figsize=(n, n))
  plt.imshow(canvas, origin="upper", cmap="gray")
  plt.show()


def draw_noise(noise_input):
  # Generate image from noise.
  guess = generator.predict(noise_input)
  draw_data(guess)

  
def draw_from_dataset(data):
  draw_data(data)


# Generate images from noise, using the generator network.
n = 6
draw_noise(generate_sliding_noise([10,20,30,40,50,60, 70, 80,90]))
z = np.random.normal(-1., 1., size=[n * n, noise_dim]).astype(np.float32)
draw_noise(z)

In [0]:
training_steps = 10000
display_step = 500


def test_images():
  z = np.random.normal(-1., 1., size=[n * n, noise_dim]).astype(np.float32)
  draw_noise(z)

# Run training for the given number of steps.
for i, batch_x in enumerate(train_data.take(training_steps)):
    
    if i % display_step == 0:
      print('On Batch ', i, '/', training_steps, '(', total_training_epochs, 'total )')
      test_images()
    
    total_training_epochs += 1
    
    # Generate noise.
    noise_input = np.random.normal(
        -1., 1., size=[batch_size, noise_dim]).astype(np.float32)
    # use noise to create fake images
    fake_images = generator.predict(noise_input)
    # train the discriminator, using both fake and real images
    discriminator.trainable = True
    discriminator.train_on_batch(fake_images, false_labels)
    discriminator.train_on_batch(batch_x, true_labels)
    
    # Generate noise.
    noise_input = np.random.normal(
        -1., 1., size=[batch_size, noise_dim]).astype(np.float32)
    discriminator.trainable = False
    combined.train_on_batch(noise_input, true_labels)


print('done')

In [0]:
save_model(generator, 'generator')
save_model(discriminator, 'discriminator')